In [1]:
%pip install requests azure-identity msgraph-sdk neo4j graphdatascience

  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached neo4j-5.25.0-py3-none-any.whl.metadata (5.7 kB)
  Using cached graphdatascience-1.12-py3-none-any.whl.metadata (7.5 kB)
  Using cached charset_normalizer-3.4.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (34 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.2.3-py3-none-any.whl.metadata (6.5 kB)
  Using cached certifi-2024.8.30-py3-none-any.whl.metadata (2.2 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached pytz-2024.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached multimethod-1.12-py3-none-any.whl.metadata (9.6 kB)
  Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached pandas-2.2.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (89 kB)
  Using cached pyarrow-16.1.0-cp312-cp312-manylinux_2_28_x86_64.whl.metadata 

In [9]:
import configparser
from msgraph.generated.models.o_data_errors.o_data_error import ODataError

from configparser import SectionProxy
from azure.identity import DeviceCodeCredential
from msgraph import GraphServiceClient
from msgraph.generated.users.item.user_item_request_builder import UserItemRequestBuilder
from msgraph.generated.sites.sites_request_builder import SitesRequestBuilder
from kiota_abstractions.base_request_configuration import RequestConfiguration

In [3]:
class Graph:
    settings: SectionProxy
    device_code_credential: DeviceCodeCredential
    user_client: GraphServiceClient

    def __init__(self,config: SectionProxy):
        self.settings = config
        client_id = self.settings['clientId']
        tenant_id = self.settings['tenantId']
        graph_scopes = self.settings['graphUserScopes'].split(' ')

        self.device_code_credential = DeviceCodeCredential(client_id, tenant_id=tenant_id)
        self.user_client = GraphServiceClient(self.device_code_credential, graph_scopes)

    async def get_user_token(self):
        graph_scopes = self.settings['GraphUserScopes']
        access_token = self.device_code_credential.get_token(graph_scopes)
        return access_token.token
    
    async def get_user(self):
        # only request specific properties using $select
        query_params = UserItemRequestBuilder.UserItemRequestBuilderGetQueryParameters(
            select=['displayName', 'mail', 'userPrincipalName']
        )
        request_config = UserItemRequestBuilder.UserItemRequestBuilderGetRequestConfiguration(
            query_parameters=query_params
        )
        user = await self.user_client.me.get(request_configuration=request_config)
        return user
    
    async def get_default_site(self):
        query_params = UserItemRequestBuilder.UserItemRequestBuilderGetQueryParameters(
            select=["siteCollection","webUrl"]
        )
        request_config = UserItemRequestBuilder.UserItemRequestBuilderGetRequestConfiguration(
            query_parameters=query_params
        )
        site = await self.user_client.sites.get(request_configuration=request_config)
        return site

In [4]:
async def greet_user(graph: Graph): 
    user = await graph.get_user()
    if user:
        print(f'Hello {user.display_name}')
        print(f'Email: {user.mail or user.user_principal_name}')

In [5]:
config = configparser.ConfigParser()
config.read(['config.cfg'])
azure_settings = config['azure']

graph: Graph = Graph(azure_settings)

In [6]:
await greet_user(graph)

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code SDTNP98MK to authenticate.
Hello Muhammad Arif Wicaksana
Email: arif@muarwi.onmicrosoft.com


In [7]:
async def get_site(graph: Graph):
    site = await graph.get_default_site()
    if site:
        print(site.display_name)

In [8]:
await get_site(graph)

AttributeError: 'SiteCollectionResponse' object has no attribute 'display_name'

In [13]:
query_params = SitesRequestBuilder.SitesRequestBuilderGetQueryParameters(
    select=['siteCollection','webUrl'],
    filter=['siteCollection/root ne null']
)

request_configuration = RequestConfiguration(
    query_parameters=query_params
)

result = await graph.sites.get(request_configuration=request_configuration)

AttributeError: 'Graph' object has no attribute 'sites'